In [2]:
import librosa
import numpy as np

In [14]:
!pip install matplotlib

In [41]:
audio_path = 'archive/Piano_Notes/piano-bb_D#_major.wav'

In [15]:
y,sr = librosa.load(audio_path)
duration = librosa.get_duration(y = y, sr = sr)
samples = len(y)

In [16]:
n_fft = 2048 # window length for short forier transform
hop_length = 512 # hop difference between each window

In [17]:
samples

33968

In [26]:
y, _ = librosa.effects.trim(y, top_db = 10)
stft = librosa.stft(y)
chromagram = librosa.feature.chroma_stft(S = np.abs(stft), sr = sr)

In [27]:
chromagram.shape

(12, 22)

In [29]:
def get_chromagram(audio_file):
    y,sr = librosa.load(audio_file)
    y, _ = librosa.effects.trim(y, top_db = 10)
    stft = librosa.stft(y)
    chromagram = librosa.feature.chroma_stft(S = np.abs(stft), sr = sr)
    return chromagram

In [43]:
audio_path = 'archive/Piano_Notes/piano-eb_D#_major.wav'

In [44]:
chromagram = get_chromagram(audio_path)

In [45]:
prominant_pitch = np.argmax(chromagram[:,21])

In [46]:
pitch_classes = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]

In [47]:
pitch_name = pitch_classes[prominant_pitch]
pitch_name

'D#'

In [5]:
# Designing the Circuit
import qiskit
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.parameter import Parameter
import numpy as np

In [19]:
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.circuit import Parameter
 
angle = Parameter("angle1")  # undefined number
 
# Create and optimize circuit once
qc = QuantumCircuit(1)
qc.ry(angle, 0)

qc = generate_preset_pass_manager(optimization_level=3, basis_gates=["u", "cx"]).run(qc)
 
qc.draw(output = 'text')

┌───────────────┐
q: ┤ U(angle1,0,0) ├
   └───────────────┘

In [8]:
def vqc(n_qubits, n_layers, n_features):
    qr = QuantumRegister(n_qubits, 'q')
    cr = ClassicalRegister(n_qubits, 'c')
    qc = QuantumCircuit(qr, cr, name = 'vqc')

    # using angle encoding
    # REASONS: direct maping of classical data
    #           using continuous variable to represent feature, to preserve mor eof the original
    for i in range(n_features):
        angle = Parameter(f'feature{i}')
        qc.ry(angle, qr[i % n_qubits])   # reusing the same qubits to code multiple features

    # for the number of circuit layers we want to implement
    for layer in range(n_layers):
        for i in range(n_qubits):
            theta = Parameter(f'theta_{layer}_{i}')
            phi = Parameter(f'phi_{layer}_{i}')
            lam = Parameter(f'lambda_{layer}_{i}')
            qc.u(theta, phi, lam, qr[i])

        for i in range(n_qubits -1):
            qc.cx(qr[i], qr[i+1])

        if n_qubits > 1: 
            qc.cx(qr[n_qubits -1], qr[0])

    qc.measure(qr,cr)
    return qc